In [41]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [42]:
Groq_api_key=os.getenv('GROQ_API_KEY')

Groq_api_key

'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'

In [43]:
from langchain_groq import ChatGroq

model=ChatGroq(model='Gemma2-9b-It', groq_api_key=Groq_api_key)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11906a420>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1190b9100>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [44]:
from langchain_core.messages import HumanMessage, SystemMessage

messages=[
    SystemMessage(content="Translate the following from english to french"),
    HumanMessage(content="Hello, How are You?")
]

result=model.invoke(messages)

result

AIMessage(content="Hello, how are you? \n\nHere are a few ways to translate that into French:\n\n**Formal:**\n\n* Bonjour, comment allez-vous ?\n\n**Informal:**\n\n* Salut, comment vas-tu ? (used with someone you know well)\n* Salut, ça va ? (very casual)\n\n\nLet me know if you have any other phrases you'd like translated!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 22, 'total_tokens': 105, 'completion_time': 0.150909091, 'prompt_time': 0.002150915, 'queue_time': 0.243772785, 'total_time': 0.153060006}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--9db5739d-db79-42fe-8bea-a755f2514e18-0', usage_metadata={'input_tokens': 22, 'output_tokens': 83, 'total_tokens': 105})

In [45]:
from langchain_core.output_parsers import StrOutputParser  # gives only content
parser=StrOutputParser()

parser.invoke(result)


"Hello, how are you? \n\nHere are a few ways to translate that into French:\n\n**Formal:**\n\n* Bonjour, comment allez-vous ?\n\n**Informal:**\n\n* Salut, comment vas-tu ? (used with someone you know well)\n* Salut, ça va ? (very casual)\n\n\nLet me know if you have any other phrases you'd like translated!\n"

In [46]:
# using LCEL -> chain the components

chain = model | parser
chain.invoke(messages)

'Here are a few ways to say "Hello, How are you?" in French:\n\n* **Formal:**\n    * Bonjour, comment allez-vous ?\n* **Informal:**\n    * Salut, comment vas-tu ? (for singular, to someone you know well)\n    * Salut, comment allez-vous ? (for plural, to multiple people, or someone you don\'t know well) \n\n\n\nLet me know if you\'d like more variations or have other phrases you want translated!\n'

In [47]:
# Prompt Templates 

from langchain_core.prompts import ChatPromptTemplate
generic_template="translate the following into {language} :"

prompt=ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
    
)

result=prompt.invoke({"language": "French", "text":"Hello"})

result

ChatPromptValue(messages=[SystemMessage(content='translate the following into French :', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [48]:
result.to_messages()

[SystemMessage(content='translate the following into French :', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [50]:
# Chaining together components with LCEL

chain=prompt | model | parser
chain.invoke({"language": "French", "text": "where are you"})

'Où êtes-vous ? \n'